In [1]:

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

from scipy.stats import mode

__Import dataset__

In [2]:
hotel_booking_dataset = pd.read_csv("hotel_bookings.csv")
features_array = hotel_booking_dataset.columns

In [3]:
print(f"The nums of dataset: {hotel_booking_dataset.shape[0]}")
print(f"The nums of feature: {hotel_booking_dataset.shape[1]}")

The nums of dataset: 119390
The nums of feature: 32


__Feature Information__

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
print(hotel_booking_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [5]:
print(hotel_booking_dataset.describe())

       is_canceled  lead_time  arrival_date_year  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights    adults  children    babies  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled  booking_changes     agent  company  days_in_waiting_list       adr  required_car_parking_spaces  total_of_special_requests
count    119390.00  119390.00          119390.00                 119390.00                  119390.00                119390.00             119390.00 119390.00 119386.00 119390.00          119390.00               119390.00                       119390.00        119390.00 103050.00  6797.00             119390.00 119390.00                    119390.00                  119390.00
mean          0.37     104.01            2016.16                     27.17                      15.80                     0.93                  2.50      1.86      0.10      0.01               0.03                    0.09                       

__Check feature missing value__

In [6]:
for i in range(len(features_array)):
    if hotel_booking_dataset[features_array[i]].isnull().any():
        missing_value = hotel_booking_dataset[features_array[i]].isnull().sum()
        print(f"Column '{features_array[i]}' contains {missing_value} null values.")

Column 'children' contains 4 null values.
Column 'country' contains 488 null values.
Column 'agent' contains 16340 null values.
Column 'company' contains 112593 null values.


__Preprocessing missing value__

In [7]:
hotel_booking_dataset["children"].fillna(value=hotel_booking_dataset["children"].mode()[0],inplace=True)
hotel_booking_dataset['children'] = hotel_booking_dataset['children'].astype('int32')
hotel_booking_dataset['country'].fillna("Unknown",inplace=True)
hotel_booking_dataset.drop(columns=['agent', 'company','reservation_status'], inplace=True)

__Normalization__

In [8]:
max_lead_time = hotel_booking_dataset['lead_time'].max()
min_lead_time = hotel_booking_dataset['lead_time'].min()

hotel_booking_dataset['lead_time'] = (hotel_booking_dataset['lead_time'] - min_lead_time)/ (max_lead_time - min_lead_time)

__Preprocessing Outlier__

In [9]:
outlier_person_type_condition = ((hotel_booking_dataset['adults'] == 0) & (hotel_booking_dataset['children'] == 0) & (hotel_booking_dataset['babies'] == 0)) | (hotel_booking_dataset['adults'] + hotel_booking_dataset['children'] + hotel_booking_dataset['babies'] > 7)
outlier_stay_type_condition = ((hotel_booking_dataset['stays_in_weekend_nights'] == 0) & hotel_booking_dataset['stays_in_week_nights'] == 0)

means = hotel_booking_dataset.mean(numeric_only=True).astype(int)
median = hotel_booking_dataset.median(numeric_only=True).astype(int)


hotel_booking_dataset.loc[outlier_person_type_condition, 'adults'] = means['adults']
hotel_booking_dataset.loc[outlier_person_type_condition, 'children'] = means['children']
hotel_booking_dataset.loc[outlier_person_type_condition, 'babies'] = means['babies']

hotel_booking_dataset.loc[outlier_stay_type_condition, 'stays_in_weekend_nights'] = median['stays_in_weekend_nights']
hotel_booking_dataset.loc[outlier_stay_type_condition, 'stays_in_week_nights'] = median['stays_in_week_nights']

__时间转换__

In [10]:
hotel_booking_dataset['reservation_status_date'] = pd.to_datetime(hotel_booking_dataset['reservation_status_date'])

hotel_booking_dataset['arrival_date_month'] = hotel_booking_dataset['reservation_status_date'].dt.month


In [11]:
hotel_booking_dataset.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,Resort Hotel,0,0.46,2015,7,27,1,1,2,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,Transient,0.00,0,0,2015-07-01
1,Resort Hotel,0,1.00,2015,7,27,1,1,2,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,Transient,0.00,0,0,2015-07-01
2,Resort Hotel,0,0.01,2015,7,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,Transient,75.00,0,0,2015-07-02
3,Resort Hotel,0,0.02,2015,7,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,0,Transient,75.00,0,0,2015-07-02
4,Resort Hotel,0,0.02,2015,7,27,1,1,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,0,Transient,98.00,0,1,2015-07-03


__非数值数据转换__

In [12]:
str_feature = ['hotel','meal','country','market_segment','distribution_channel','reserved_room_type','assigned_room_type','customer_type']

encoded_features_df = pd.get_dummies(hotel_booking_dataset[str_feature], drop_first=True)
new_hotel_booking_dataset = hotel_booking_dataset.drop(columns=str_feature)

new_hotel_booking_dataset = pd.concat([new_hotel_booking_dataset, encoded_features_df], axis=1)

new_hotel_booking_dataset.head(50)


,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,hotel_Resort Hotel,meal_FB,meal_HB,meal_SC,meal_Undefined,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,country_ATF,country_AUS,country_AUT,country_AZE,country_BDI,country_BEL,country_BEN,country_BFA,country_BGD,country_BGR,country_BHR,country_BHS,country_BIH,country_BLR,country_BOL,country_BRA,country_BRB,country_BWA,country_CAF,country_CHE,country_CHL,country_CHN,country_CIV,country_CMR,country_CN,country_COL,country_COM,country_CPV,country_CRI,country_CUB,country_CYM,country_CYP,country_CZE,country_DEU,country_DJI,country_DMA,country_DNK,country_DOM,country_DZA,country_ECU,country_EGY,country_ESP,country_EST,country_ETH,country_FIN,country_FJI,country_FRA,country_FRO,country_GAB,country_GBR,country_GEO,country_GGY,country_GHA,country_GIB,country_GLP,country_GNB,country_GRC,country_GTM,country_GUY,country_HKG,country_HND,country_HRV,country_HUN,country_IDN,country_IMN,country_IND,country_IRL,country_IRN,country_IRQ,country_ISL,country_ISR,country_ITA,country_JAM,country_JEY,country_JOR,country_JPN,country_KAZ,country_KEN,country_KHM,country_KIR,country_KNA,country_KOR,country_KWT,country_LAO,country_LBN,country_LBY,country_LCA,country_LIE,country_LKA,country_LTU,country_LUX,country_LVA,country_MAC,country_MAR,country_MCO,country_MDG,country_MDV,country_MEX,country_MKD,country_MLI,country_MLT,country_MMR,country_MNE,country_MOZ,country_MRT,country_MUS,country_MWI,country_MYS,country_MYT,country_NAM,country_NCL,country_NGA,country_NIC,country_NLD,country_NOR,country_NPL,country_NZL,country_OMN,country_PAK,country_PAN,country_PER,country_PHL,country_PLW,country_POL,country_PRI,country_PRT,country_PRY,country_PYF,country_QAT,country_ROU,country_RUS,country_RWA,country_SAU,country_SDN,country_SEN,country_SGP,country_SLE,country_SLV,country_SMR,country_SRB,country_STP,country_SUR,country_SVK,country_SVN,country_SWE,country_SYC,country_SYR,country_TGO,country_THA,country_TJK,country_TMP,country_TUN,country_TUR,country_TWN,country_TZA,country_UGA,country_UKR,country_UMI,country_URY,country_USA,country_UZB,country_Unknown,country_VEN,country_VGB,country_VNM,country_ZAF,country_ZMB,country_ZWE,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0,0.46,2015,7,27,1,1,2,2,0,0,0,0,0,3,No Deposit,0,0.00,0,0,2015-07-01,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [13]:
print(new_hotel_booking_dataset.shape)

(119390, 237)


__Decision Tree based on gini index__

In [14]:
X = new_hotel_booking_dataset.drop(columns=['is_canceled','reservation_status_date','deposit_type'])
y = new_hotel_booking_dataset['is_canceled']

X_train, X_test, Y_train, Y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [16]:


class RandomForest:
    def __init__(self, max_depth, forest_size, max_features = 'auto'):
        self.max_depth = max_depth
        self.forest_size = forest_size
        self.trees = []

        for _ in range(forest_size):
            clf = tree.DecisionTreeClassifier(criterion='gini',splitter='random',max_depth=self.max_depth, max_features=max_features)
            clf.fit(X=X_train,y=Y_train)
            self.trees.append()
    
    def predict(self, X):
        predictions = []
        for the_tree in self.trees:
            predictions.append(the_tree.predict(X))
        predictions = np.stack(predictions)
        result = np.array(mode(predictions, axis=0)[0])
        return result


[1 0 1 0]


__Data Analysis__